In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# Define the URL of the page to scrape
url = "https://pih.com.pk/doctors"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Define the doctor types of interest
doctor_types = [
    "PSYCHIATRIST",
    "NEUROLOGIST",
    "DEVELOPMENTAL PEDIATRICIAN",
    "CLINICAL PSYCHOLOGIST",
    "OCCUPATIONAL THERAPIST"
]

# Initialize an empty list to store doctor data
doctor_data = []

# Find all doctor sections in the HTML
for section in soup.find_all("div", class_="elementor-column"):
    # Extract the doctor's type
    doctor_type_elem = section.find("h6", class_="elementor-heading-title")
    if doctor_type_elem:
        doctor_type = doctor_type_elem.get_text(strip=True)
        if doctor_type.upper() in doctor_types:
            # Extract doctor's name
            name_elem = section.find("h3", class_="elementor-heading-title")
            name = name_elem.get_text(strip=True) if name_elem else "N/A"
            
            # Extract doctor's image URL
            img_elem = section.find("img")
            img_url = img_elem['src'] if img_elem else "N/A"
            
            # Extract other details
            details_elem = section.find("p")
            details = details_elem.get_text(strip=True) if details_elem else "N/A"
            
            # Parse the details to extract specific information
            lines = details.split('\n')
            education = lines[0].strip() if len(lines) > 0 else "N/A"
            position = lines[1].strip() if len(lines) > 1 else "N/A"
            
            # Assuming timing details are in a fixed format
            timing_details = lines[2].strip() if len(lines) > 2 else "N/A"
            days = []
            timing = {}
            if 'Mon' in timing_details:
                days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
                timing = {
                    "Monday": timing_details.split("–")[-1].strip(),
                    "Tuesday": timing_details.split("–")[-1].strip(),
                    "Wednesday": timing_details.split("–")[-1].strip(),
                    "Thursday": timing_details.split("–")[-1].strip(),
                    "Friday": timing_details.split("–")[-1].strip(),
                    "Saturday": timing_details.split("–")[-1].strip()
                }
            
            # Add the doctor data to the list
            doctor_data.append({
                "name": name,
                "education": education,
                "hospital": "Not specified",
                "position": position,
                "clinic": {
                    "days": days,
                    "timing": timing
                }
            })

# Print the scraped doctor data
print(json.dumps(doctor_data, indent=4))
